In [3]:
########################
###### TIDY :) #########
########################

import requests
from bs4 import BeautifulSoup

def scrape_three_letter():
    """
    Returns a list of three letter country codes from the hardcoded website.
    url is hardcoded because the scraping is specific to the site.
    <h4 class="margin-clear">ALB</h4>
    
    scrape_three_letter() ==> ['AFG','ALA','ALB','DZA',]
    """
    url = 'https://laendercode.net/en/3-letter-list.html'
    
    try:
        response = requests.get(url)
        assert response.status_code == 200
    except:
        print(f'response code: {response.status_code}')  
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    country_abv = [el.text for el in soup.find_all(class_='margin-clear')]
    return(country_abv)

tlc_list = scrape_three_letter()


In [6]:
import pandas as pd
import numpy as np
#second website with country data, needs 3-letter-code added to url

########################
###### TIDY :) #########
########################


def process_country(a_country):
    """
    process_country(a_country) ==> DataFrame (df.shape() ==> 4 rows x 59 columns)
    Scrapes worldbank.org website for country related data starting with an input three letter
    country code as string. The website has 4 years of data for a country, all data is retreived. 
    This returns a pandas dataframe with 4 rows. 
    """
    #make beautifulsoup object from web data for a_country
    url_1 = f'http://databank.worldbank.org/data/views/reports/reportwidget.aspx?Report_Name=CountryProfile&Id=b450fd57&tbar=y&dd=y&inf=n&zm=n&country='
    url_2 = url_1 + a_country
    response = requests.get(url_2)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    #retreive year info from headers
    header_table=soup.find(id="table-ddpreport-header")
    years = [x.text for x in header_table.find_all(class_="spacer2")]    
    
    #retreive data
    data_table = soup.find(id="table-ddpreport-data")        
    rows = data_table.find_all(True, {"class":["rowdata level-0 ", "rowdata level-0 alternate"]})
    
    #Construct dataframe object COUTNRY YEAR F1 F2 F3...
    #I need to double check that this assembles the table correctly (compare to web data)
    df = pd.DataFrame()
    for row in rows: 
        row_data = [x.text for x in row.find_all('td')]
        df[row_data[0]] = row_data[1:]
    df.insert(0,'COUNTRY',a_country)
    df.insert(1,'YEAR',years)
    return df

#simple test using a country code provided from previous cell
a_country = tlc_list[0]
process_country(a_country)



,COUNTRY,YEAR,"Population, total (millions)",Population growth (annual %),Surface area (sq. km) (thousands),Population density (people per sq. km of land area),Poverty headcount ratio at national poverty lines (% of population),Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),"GNI, Atlas method (current US$) (billions)","GNI per capita, Atlas method (current US$)",...,High-technology exports (% of manufactured exports),Statistical Capacity score (Overall average),Merchandise trade (% of GDP),Net barter terms of trade index (2000 = 100),"External debt stocks, total (DOD, current US$) (millions)","Total debt service (% of exports of goods, services and primary income)",Net migration (thousands),"Personal remittances, received (current US$) (millions)","Foreign direct investment, net inflows (BoP, current US$) (millions)",Net official development assistance received (current US$) (millions)
0,AFG,1990,12.25,3.9,652.9,18.8,..,..,..,..,...,..,..,..,..,0,..,"2,327",..,0,121.7
1,AFG,2000,20.09,3.5,652.9,30.8,..,..,..,..,...,..,..,72,100,0,..,929,..,0,136.0
2,AFG,2010,28.80,2.8,652.9,44.1,35.8,..,14.43,500,...,..,52,35,145,"2,425",0.4,448,362,191,"6,470.4"
3,AFG,2017,35.53,2.5,652.9,54.4,..,..,20.18,570,...,..,51,41,161,"2,404",3.5,-300,378,53,"4,064.3"


In [8]:
########################
###### TIDY :) #########
########################

def process_world(list_three_letter_abr):
    """
    uses webscraper helper function process_country() to create individual dataframes 
    which are appended into a large dataframe. 
    
    process_world(list_three_letter_abr) ==> df
    """
    total=pd.DataFrame() 

    for x in list_three_letter_abr : #remove slicing to generate whole list
        total = total.append(process_country(x))
    
    #after generating the whole set, relabel the indexes    
    total.index = range(total.shape[0])
    return total

#short test for function above
#pw1 = process_world(tlc_list[0:2])

#for all countries use
pw1 = process_world(tlc_list)

In [120]:
#mapping dictionary for renaming columns
nick = {'COUNTRY': 'CNT',
'YEAR': 'YR',              
'Population, total (millions)': 'POP_TOT' ,        
'Population growth (annual %)': 'POP_GRO%',        
'Surface area (sq. km) (thousands)': 'AREA',            
'Population density (people per sq. km of land area)': 'POP_DEN',         
'Poverty headcount ratio at national poverty lines (% of population)': 'POV',            
'Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)': 'POV_190', 
'GNI, Atlas method (current US$) (billions)': 'GNI_ATL',
'GNI per capita, Atlas method (current US$)': 'GPC_ATL',
'GNI, PPP (current international $) (billions)': 'GNI_INT',
'GNI per capita, PPP (current international $)': 'GNI_PPP',
'Income share held by lowest 20%': 'INC_LOW20', 
'Life expectancy at birth, total (years)': 'LIF_EXP',
'Fertility rate, total (births per woman)': 'FER_RATE',
'Adolescent fertility rate (births per 1,000 women ages 15-19)': 'TEEN_FER_RATE',
'Contraceptive prevalence, any methods (% of women ages 15-49)': 'CONTR',
'Births attended by skilled health staff (% of total)': 'HOSP_BIRTH',
'Mortality rate, under-5 (per 1,000 live births)': 'MORT_RATE',
'Prevalence of underweight, weight for age (% of children under 5)'	: 'UNDER_WGT_BIRTH', 
'Immunization, measles (% of children ages 12-23 months)': 'IMM_MEAS',
'Primary completion rate, total (% of relevant age group)': 'ELEM_GRAD',
'School enrollment, primary (% gross)': 'SCH_ENR_PR',
'School enrollment, secondary (% gross)': 'SCH_ENR_SEC',
'School enrollment, primary and secondary (gross), gender parity index (GPI)': 'SCH_ENR_PR&SEC',
'Prevalence of HIV, total (% of population ages 15-49)': 'HIV',
'Forest area (sq. km) (thousands)': 'FORR_AREA',
'Terrestrial and marine protected areas (% of total territorial area)': 'PROT_AREA%',
'Annual freshwater withdrawals, total (% of internal resources)': 'FRESH_WITH%',
'Urban population growth (annual %)': 'URB_GRO%',
'Energy use (kg of oil equivalent per capita)': 'ENRG_PERCAP',
'CO2 emissions (metric tons per capita)': 'CO2_PERCAP',
'Electric power consumption (kWh per capita)': 'ELEC_PERCAP',
'GDP (current US$) (billions)': 'GDP',
'GDP growth (annual %)': 'GDP_GROW%',
'Inflation, GDP deflator (annual %)': 'INF%',
'Agriculture, forestry, and fishing, value added (% of GDP)': 'AG_FOR_FISH%GDP',
'Industry (including construction), value added (% of GDP)': 'IND_GDP',
'Exports of goods and services (% of GDP)': 'EXP%GDP',
'Imports of goods and services (% of GDP)': 'IMP%GDP',
'Gross capital formation (% of GDP)': 'GCF%GDP',
'Revenue, excluding grants (% of GDP)': 'REV_NO_GRANT%GDP',
'Net lending (+) / net borrowing (-) (% of GDP)': 'LOANS%GDP',
'Time required to start a business (days)': 'TIME_START_BUS',
'Domestic credit provided by financial sector (% of GDP)': 'DOM_CRED%GDP',
'Tax revenue (% of GDP)': 'TAX_REV%GDP',
'Military expenditure (% of GDP)': 'MILIT%GDP',
'Mobile cellular subscriptions (per 100 people)': 'CELL_SUB%',
'Individuals using the Internet (% of population)': 'INTERNET%',
'High-technology exports (% of manufactured exports)': 'HIGH_TECH_EXP%',
'Statistical Capacity score (Overall average)': 'STAT_CAP',
'Merchandise trade (% of GDP)': 'MERCH%GDP',
'Net barter terms of trade index (2000 = 100)': 'NET_BART',
'External debt stocks, total (DOD, current US$) (millions)'	: 'EXT_DEBT_STOCK',
'Total debt service (% of exports of goods, services and primary income)': 'TOT_SEBT_SERV%',
'Net migration (thousands)': 'NET_MIG',
'Personal remittances, received (current US$) (millions)': 'REMIT',
'Foreign direct investment, net inflows (BoP, current US$) (millions)' : 'FOR_INV_NET',
'Net official development assistance received (current US$) (millions)' : 'OFF_DEV_ASS_NET'}

In [167]:
########################
###### TIDY :) #########
########################

def clean_df(df):
    """
    data entries include commas in numbers (need to remove), and '..' for data which is NA 
    (should replace with NaN). Also change number data to float and change column headers 
    to shortened nicknames. And set the column types correctly. And shorten the names of columns.

    clean_df(df) ==> df
    """
    #ensure that there are no references to original df
    dfc = df.copy()
    #shorter names for columns
    dfc = dfc.rename(index=str, columns=nick)
    #convert NA indicator to NaN
    dfc[dfc == '..'] = np.nan
    #protect the CNT column (0) from to_numeric conversion
    dfc = dfc.applymap(lambda x: str(x).replace(',', ''))
    dfc.iloc[:,1:]=dfc.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
    #need infer_objects() to convert the column types automatically now that all 'nan' ==> NaN
    dfc=dfc.infer_objects()
    return dfc

cdf = clean_df(pw1)

# Questions On Cleanup

```
#changes all columns to numeric type, including country names ('CNT') now all NaN
#dfc.apply(pd.to_numeric, errors='coerce')

#changes all columns to numeric type except 'CNT' (still object)
#country_col = dfc.CNT
#dfc=dfc.apply(pd.to_numeric, errors='coerce')
#dfc['CNT'] = country_col

#changes string nan to NaN, but does not change the type of any columns (all object)
#dfc.iloc[:,1:]=dfc.iloc[:,1:].apply(pd.to_numeric, errors='coerce')

#need infer_objects() to convert the column types automatically now that all 'nan' ==> NaN
#dfc.iloc[:,1:]=dfc.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
#dfc=dfc.infer_objects()
```

In [278]:
cdf.head()

,CNT,YR,POP_TOT,POP_GRO%,AREA,POP_DEN,POV,POV_190,GNI_ATL,GPC_ATL,...,HIGH_TECH_EXP%,STAT_CAP,MERCH%GDP,NET_BART,EXT_DEBT_STOCK,TOT_SEBT_SERV%,NET_MIG,REMIT,FOR_INV_NET,OFF_DEV_ASS_NET
0,AFG,1990.0,12.25,3.9,652.9,18.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,2327.0,NaN,0.0,121.7
1,AFG,2000.0,20.09,3.5,652.9,30.8,NaN,NaN,NaN,NaN,...,NaN,NaN,72.0,100.0,0.0,NaN,929.0,NaN,0.0,136.0
2,AFG,2010.0,28.80,2.8,652.9,44.1,35.8,NaN,14.43,500.0,...,NaN,52.0,35.0,145.0,2425.0,0.4,448.0,362.0,191.0,6470.4
3,AFG,2017.0,35.53,2.5,652.9,54.4,NaN,NaN,20.18,570.0,...,NaN,51.0,41.0,161.0,2404.0,3.5,-300.0,378.0,53.0,4064.3
4,ALA,1990.0,5288.10,1.7,134042.4,40.8,NaN,35.9,22236.07,4205.0,...,18.0,NaN,31.0,NaN,NaN,NaN,0.0,68441.0,196315.0,58242.2


In [281]:
#need to decide whether to remove the NaN cells???!?!??!
cdf.shape
cdf.to_pickle("./cleaned_with_NaN.pkl")

In [236]:
ddf = cdf.copy()

In [181]:
filtered_ddf = ddf[ddf.notnull()]

In [195]:
data_ddf = pd.DataFrame(ddf.isna().sum())

In [277]:
ddf3 = ddf.dropna(thresh=800,axis=1).dropna(thresh=18)

In [275]:
ddf3

,CNT,YR,POP_TOT,POP_GRO%,AREA,POP_DEN,LIF_EXP,FER_RATE,TEEN_FER_RATE,MORT_RATE,...,FORR_AREA,URB_GRO%,CO2_PERCAP,GDP,GDP_GROW%,INF%,CELL_SUB%,INTERNET%,NET_MIG,FOR_INV_NET
1,AFG,2000.0,20.09,3.5,652.9,30.8,55.0,7.5,154.0,129.0,...,13.5,3.9,0.04,2.46,8.4,11.7,0.0,0.0,929.0,0.0
2,AFG,2010.0,28.80,2.8,652.9,44.1,61.0,5.8,99.0,90.0,...,13.5,3.7,0.29,15.94,8.4,9.4,35.5,4.0,448.0,191.0
3,AFG,2017.0,35.53,2.5,652.9,54.4,64.0,4.6,69.0,68.0,...,13.5,3.4,0.30,20.82,2.6,4.9,67.4,10.6,-300.0,53.0
4,ALA,1990.0,5288.10,1.7,134042.4,40.8,65.0,3.3,65.0,93.0,...,41282.7,2.6,4.19,22573.78,2.9,7.6,0.2,0.0,0.0,196315.0
5,ALA,2000.0,6121.68,1.3,134112.3,47.2,68.0,2.7,56.0,77.0,...,40556.0,2.2,4.03,33571.15,4.4,4.5,12.1,6.7,0.0,1460994.0
6,ALA,2010.0,6932.87,1.2,134311.4,53.4,71.0,2.5,47.0,52.0,...,40156.7,2.2,4.83,65956.67,4.3,4.4,76.1,28.7,0.0,1860424.0
7,ALA,2017.0,7530.36,1.2,134325.1,58.0,72.0,2.4,45.0,39.0,...,39991.3,2.0,4.97,80683.79,3.2,3.4,104.5,45.8,0.0,1862732.0
8,ALB,1990.0,3.29,1.8,28.8,119.9,72.0,3.0,18.0,40.0,...,7.9,2.5,1.68,2.10,-9.6,-0.5,0.0,0.0,-443.0,20.0
9,ALB,2000.0,3.09,-0.6,28.8,112.7,74.0,2.2,19.0,25.0,...,7.7,0.7,0.98,3.63,6.7,4.0,1.0,0.1,-177.0,143.0
10,ALB,2010.0,2.91,-0.5,28.8,106.3,77.0,1.7,20.0,12.0,...,7.8,1.6,1.58,11.93,3.7,4.5,91.6,45.0,-93.0,1090.0


In [269]:
ddf3 = ddf2.dropna(thresh=21)

In [270]:
ddf3

,CNT,YR,POP_TOT,POP_GRO%,AREA,POP_DEN,LIF_EXP,FER_RATE,TEEN_FER_RATE,MORT_RATE,...,FORR_AREA,URB_GRO%,CO2_PERCAP,GDP,GDP_GROW%,INF%,CELL_SUB%,INTERNET%,NET_MIG,FOR_INV_NET
1,AFG,2000.0,20.09,3.5,652.9,30.8,55.0,7.5,154.0,129.0,...,13.5,3.9,0.04,2.46,8.4,11.7,0.0,0.0,929.0,0.0
2,AFG,2010.0,28.80,2.8,652.9,44.1,61.0,5.8,99.0,90.0,...,13.5,3.7,0.29,15.94,8.4,9.4,35.5,4.0,448.0,191.0
3,AFG,2017.0,35.53,2.5,652.9,54.4,64.0,4.6,69.0,68.0,...,13.5,3.4,0.30,20.82,2.6,4.9,67.4,10.6,-300.0,53.0
4,ALA,1990.0,5288.10,1.7,134042.4,40.8,65.0,3.3,65.0,93.0,...,41282.7,2.6,4.19,22573.78,2.9,7.6,0.2,0.0,0.0,196315.0
5,ALA,2000.0,6121.68,1.3,134112.3,47.2,68.0,2.7,56.0,77.0,...,40556.0,2.2,4.03,33571.15,4.4,4.5,12.1,6.7,0.0,1460994.0
6,ALA,2010.0,6932.87,1.2,134311.4,53.4,71.0,2.5,47.0,52.0,...,40156.7,2.2,4.83,65956.67,4.3,4.4,76.1,28.7,0.0,1860424.0
7,ALA,2017.0,7530.36,1.2,134325.1,58.0,72.0,2.4,45.0,39.0,...,39991.3,2.0,4.97,80683.79,3.2,3.4,104.5,45.8,0.0,1862732.0
8,ALB,1990.0,3.29,1.8,28.8,119.9,72.0,3.0,18.0,40.0,...,7.9,2.5,1.68,2.10,-9.6,-0.5,0.0,0.0,-443.0,20.0
9,ALB,2000.0,3.09,-0.6,28.8,112.7,74.0,2.2,19.0,25.0,...,7.7,0.7,0.98,3.63,6.7,4.0,1.0,0.1,-177.0,143.0
10,ALB,2010.0,2.91,-0.5,28.8,106.3,77.0,1.7,20.0,12.0,...,7.8,1.6,1.58,11.93,3.7,4.5,91.6,45.0,-93.0,1090.0


In [207]:
ddf_drop = ddf.dropna(thresh=1); ddf_drop

,CNT,YR,POP_TOT,POP_GRO%,AREA,POP_DEN,POV,POV_190,GNI_ATL,GPC_ATL,...,HIGH_TECH_EXP%,STAT_CAP,MERCH%GDP,NET_BART,EXT_DEBT_STOCK,TOT_SEBT_SERV%,NET_MIG,REMIT,FOR_INV_NET,OFF_DEV_ASS_NET
0,AFG,1990.0,12.25,3.9,652.9,18.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,2327.0,NaN,0.0,121.7
1,AFG,2000.0,20.09,3.5,652.9,30.8,NaN,NaN,NaN,NaN,...,NaN,NaN,72.0,100.0,0.0,NaN,929.0,NaN,0.0,136.0
2,AFG,2010.0,28.80,2.8,652.9,44.1,35.8,NaN,14.43,500.0,...,NaN,52.0,35.0,145.0,2425.0,0.4,448.0,362.0,191.0,6470.4
3,AFG,2017.0,35.53,2.5,652.9,54.4,NaN,NaN,20.18,570.0,...,NaN,51.0,41.0,161.0,2404.0,3.5,-300.0,378.0,53.0,4064.3
4,ALA,1990.0,5288.10,1.7,134042.4,40.8,NaN,35.9,22236.07,4205.0,...,18.0,NaN,31.0,NaN,NaN,NaN,0.0,68441.0,196315.0,58242.2
5,ALA,2000.0,6121.68,1.3,134112.3,47.2,NaN,28.6,33517.78,5475.0,...,24.0,NaN,39.0,NaN,NaN,NaN,0.0,121611.0,1460994.0,49803.6
6,ALA,2010.0,6932.87,1.2,134311.4,53.4,NaN,15.7,65033.46,9380.0,...,17.0,NaN,47.0,NaN,NaN,NaN,0.0,417811.0,1860424.0,130685.4
7,ALA,2017.0,7530.36,1.2,134325.1,58.0,NaN,10.0,78060.79,10366.0,...,18.0,NaN,52.0,NaN,NaN,NaN,0.0,573286.0,1862732.0,157676.2
8,ALB,1990.0,3.29,1.8,28.8,119.9,NaN,NaN,2.22,680.0,...,NaN,NaN,29.0,NaN,0.0,4.3,-443.0,152.0,20.0,11.1
9,ALB,2000.0,3.09,-0.6,28.8,112.7,25.4,2.0,3.60,1170.0,...,1.0,NaN,37.0,100.0,1122.0,4.5,-177.0,598.0,143.0,317.9


In [73]:
cdf[cdf == np.nan]

,CNT,YR,POP_TOT,POP_GRO%,AREA,POP_DEN,POV%,POV_190%,GNI_ATL,GPC_ATL,...,HIGH_TECH_EXP%,STAT_CAP,MERCH%GDP,NET_BART,EXT_DEBT_STOCK,TOT_SEBT_SERV%,NET_MIG,REMIT,FOR_INV_NET,Net official development assistance received (current US$) (millions)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
d_df.shape

(1008, 59)